In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import random
from shutil import copy, rmtree, make_archive
import cv2

import tqdm

random.seed(0)
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
os.mkdir('Dataset')
o_path = 'Dataset'
o_paths = {}
for i in ('train', 'test', 'val'):
    os.mkdir(os.path.join(o_path, i))
    o_paths[i] = os.path.join(o_path, i)
    

In [5]:
def create_training_set(output_path, path = '../input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train'):
    classes = os.listdir(path)
    for cls in tqdm.tqdm(classes):
        class_path = os.path.join(path, cls)
        img_names = list(os.listdir(class_path))
        random.shuffle(img_names)

        # Create folder
        temp_path = os.path.join(output_path, cls)
        try:
            os.mkdir(temp_path)
        except:
            rmtree(temp_path)
            os.mkdir(temp_path)
        
        for img in img_names[:100]:
            src_path = os.path.join(class_path, img)
            dst_path = os.path.join(temp_path, img)
            cv2.imwrite(dst_path, cv2.resize(cv2.imread(src_path), (224,224)))

In [6]:
create_training_set(o_paths['train'])

100%|██████████| 1000/1000 [35:13<00:00,  2.11s/it]


In [7]:
import xml.etree.ElementTree as ET
def create_val_set(
    output_path, 
    data_path = '../input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/val', 
    annotation_path = '../input/imagenet-object-localization-challenge/ILSVRC/Annotations/CLS-LOC/val'):
    cls_path = set()
    for i in tqdm.tqdm(range(1, 50001)):
        name = f'ILSVRC2012_val_{i:08}'
        
        # Step 1 get class
        
        
        cls = ET.parse(os.path.join(annotation_path, name + '.xml')).find('object').find('name').text
        
        # Step 2 check if class exists and if not create folder
        output_folder = os.path.join(output_path, cls)
        if cls not in cls_path:
            try:
                os.mkdir(output_folder)
            except:
                rmtree(output_folder)
                os.mkdir(output_folder)    
            cls_path.add(cls)
        
        # Step 3 if class exists create src path and destination path
        src_path = os.path.join(data_path, name + '.JPEG')
        dst_path = os.path.join(output_folder, name + '.jpg')
        
        # Step 4 Save image
        cv2.imwrite(dst_path, cv2.resize(cv2.imread(src_path), (224,224)))
        
        

    

In [8]:
create_val_set(o_paths['val'])


100%|██████████| 50000/50000 [24:52<00:00, 33.50it/s]


In [9]:
import tarfile
import os.path

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

In [10]:
make_tarfile('dataset.tar.gz', 'Dataset')

In [11]:
from IPython.display import FileLink

FileLink(r'dataset.tar.gz')

/kaggle/working/dataset.tar.gz